In [ ]:
library(Seurat)
library(dplyr)
library(patchwork)
# library(readr)
library(ggplot2)
#有云服务器的，可开启并运算，这里我用4个线程：
library(future)
library(qs)
# check the current active plan
plan()
# change the current plan to access parallelization
plan("multisession", workers =40)
plan()

#设置可用的内存
options(future.globals.maxSize = 4 * 1024^3)
plan("sequential")
future::plan()


### ————————合并需要分析的细胞类型————————

In [ ]:
# # 读取数据
# Endothelial <- qread("/home/user/ALI/Endothelial/Outdata/Sub_annotion.qs")
# Epithelial <- qread("/home/user/ALI/Epithelial/Outdata/Sub_annotion.qs")
# Fibroblast <- qread("/home/user/ALI/Fibroblast/Outdata/Sub_annotion.qs")

# NK_cells <- qread("/home/user/ALI/NK cells/Outdata/Sub_annotion.qs")
# Neutrophils <- qread("/home/user/ALI/Neutrophils/Outdata/Sub_annotion.qs")
# Macrophages <- qread("/home/user/ALI/Macrophages/Outdata/Sub_annotion.qs")

# # 挑选指定 celltype
# endo_sub <- subset(Endothelial, subset = celltype %in% c("Endothelial_c3", "Endothelial_c4"))
# epi_sub  <- subset(Epithelial, subset = celltype %in% c("Epithelial_c1","Epithelial_c10"))
# fib_sub  <- subset(Fibroblast, subset = celltype %in% c("Fibroblast_c0", "Fibroblast_c9", "Fibroblast_c10"))

# nk_sub <- subset(NK_cells, subset = celltype %in% c("NK_c1", "NK_c5"))
# neu_sub  <- subset(Neutrophils, subset = celltype %in% c("Neutrophils_c2","Neutrophils_c5"))
# mac_sub  <- subset(Macrophages, subset = celltype %in% c("Macrophages_c1", "Macrophages_c2", "Macrophages_c7", "Macrophages_c9", "Macrophages_c10","Macrophages_c11", "Macrophages_c12"))

# # 合并为一个新的 Seurat 对象
# seurat.data <- merge(endo_sub, y = list(epi_sub, fib_sub,nk_sub,neu_sub,mac_sub), project = "ALI_Selected")

# # 检查结果
# table(seurat.data$celltype)


In [ ]:
# seurat.data = qread("Combined_Selected_Celltypes.qs")

In [ ]:
# seurat.data <- NormalizeData(object = seurat.data, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
# 2.5 保存数据
# qsave(seurat.data, file = "Combined_Selected_Celltypes.qs")

### ————————合并需要分析的细胞类型————————

## Step1. 构建cellchat对象

In [ ]:
#remotes::install_github("sqjin/CellChat")
library(CellChat)
library(patchwork)
library(Seurat)
# library(SeuratData)
library(qs)
# AvailableData()
# InstallData("pbmc3k")

In [ ]:
seurat.data = qread("Combined_Selected_Celltypes.qs")

In [ ]:
table(seurat.data@meta.data$celltype)

In [ ]:
## 1.2 构建cellchat对象
#pbmc3k里的seurat_annotations有一些NA注释，过滤掉
data.input = seurat.data@assays$RNA@data
meta.data =  seurat.data@meta.data
meta.data = meta.data[!is.na(meta.data$celltype),]
data.input = data.input[,row.names(meta.data)]

#设置因子水平
meta.data$celltype = factor(meta.data$celltype,
                                    #   levels = c("Epithelial_c0", "Epithelial_c1", "Epithelial_c2", "Epithelial_c3", "Epithelial_c4", 
                                    #                 "Epithelial_c5", "Epithelial_c6", "Epithelial_c7", "Epithelial_c8", "Epithelial_c9")
                                                    )

### 1.3 Create a CellChat object
cellchat <- createCellChat(object = data.input, 
                           meta = meta.data, 
                           group.by = "celltype")

In [ ]:
###可在cellchat对象的meta插槽中添加表型信息
# 添加meta.data信息
cellchat <- addMeta(cellchat, meta = meta.data)

# 设置默认的labels
levels(cellchat@idents) # show factor levels of the cell labels
# cellchat <- setIdent(cellchat, ident.use = "new.labels") 
# groupSize <- as.numeric(table(cellchat@idents)) # number of cells in each cell group

## Step2. 加载CellChatDB数据库

In [ ]:
### 1.4 加载CellChat受配体数据库
# CellChatDB <- CellChatDB.human # use CellChatDB.mouse if running on mouse data
CellChatDB <- CellChatDB.mouse # use CellChatDB.mouse if running on mouse data
showDatabaseCategory(CellChatDB)

In [ ]:
# Show the structure of the database
dplyr::glimpse(CellChatDB$interaction)

In [ ]:

# Fix "H2-BI" and "H2-Ea-ps" typo in CellChatDB.mouse
CellChatDB$interaction["H2-BI_KIR3DL1","interaction_name"] = "H2-BL_KIR3DL1"
CellChatDB$interaction["H2-BI_KIR3DL1","ligand"] = "H2-Bl"
CellChatDB$interaction["H2-BI_KIR3DL1","interaction_name_2"] = "H2-bl - Kir3dl1"
rownames(CellChatDB$interaction)[rownames(CellChatDB$interaction) == "H2-BI_KIR3DL1"] = "H2-BL_KIR3DL1"

CellChatDB$interaction["H2-EA-PS_CD4","interaction_name"] = "H2-EA_CD4"
CellChatDB$interaction["H2-EA-PS_CD4","ligand"] = "H2-Ea"
CellChatDB$interaction["H2-EA-PS_CD4","interaction_name_2"] = "H2-ea - Cd4"
rownames(CellChatDB$interaction)[rownames(CellChatDB$interaction) == "H2-EA-PS_CD4"] = "H2-EA_CD4"

In [ ]:
# use a subset of CellChatDB for cell-cell communication analysis
#CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling") # use Secreted Signaling
CellChatDB.use <- CellChatDB # simply use the default CellChatDB
cellchat@DB <- CellChatDB.use

## Step3. 对表达数据进行预处理

In [ ]:
cellchat <- subsetData(cellchat) # This step is necessary even if using the whole database
future::plan("multisession", workers = 2) # do parallel


# cellchat <- updateCellChat(cellchat) 如果报错Error in data.use[RsubunitsV, ] : subscript out of bounds)，执行这行代码

cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)


## Step4. 计算通讯概率，推断细胞通讯网络

In [ ]:
cellchat <- computeCommunProb(cellchat,population.size = F)
# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat <- filterCommunication(cellchat, min.cells = 10)

## Step5. 提取预测的细胞通讯网络为data frame

In [ ]:
head(cellchat@meta)

In [ ]:
# 作者提供了函数subsetCommunication去提取用户感兴趣的配受体对强度为data.frame：
#获取所有的配受体对以及其通讯概率
df.net <- subsetCommunication(cellchat)
head(df.net)

In [ ]:
#以通路为单位提取通讯信息
df.pathway = subsetCommunication(cellchat,slot.name = "netP")
head(df.pathway)

In [ ]:
levels(cellchat@idents)

In [ ]:
# 对感兴趣的细胞提取受配体信息
# 这里的 source.use = c(1) 指的是Naive CD4 T，2和3分别对应Memory CD4 T和CD14+ Mono：
df.net.sub <- subsetCommunication(cellchat, sources.use = c(1,2), targets.use = c(3))
head(df.net.sub)

In [ ]:
# 对感兴趣的通路提取受配体信息
df.net.sub <- subsetCommunication(cellchat, signaling = c("CCL", "CXCL"))
head(df.net.sub)

In [ ]:
qsave(cellchat, file = "CellChat_Res.qs")

## Step6. 在信号通路水平推断细胞通讯

In [ ]:
cellchat <- computeCommunProbPathway(cellchat)
head(cellchat@net)

In [ ]:
head(cellchat@netP)

## Step7. 计算加和的cell-cell通讯网络

In [ ]:
cellchat <- aggregateNet(cellchat)

## 绘图

In [ ]:
#remotes::install_github("sqjin/CellChat")
library(CellChat)
library(patchwork)
library(Seurat)
# library(SeuratData)
library(qs)
# AvailableData()
# InstallData("pbmc3k")

In [ ]:
cellchat = qread(file = "CellChat_Res_2.qs")

In [ ]:
cellchat@net$count

In [ ]:
groupSize <- as.numeric(table(cellchat@idents))
par(mfrow = c(1,2), xpd=TRUE)
p1 = netVisual_circle(cellchat@net$count, vertex.weight = groupSize,
                 weight.scale = T, label.edge= F,
                 title.name = "Number of interactions",
                 vertex.label.cex = 0.6)
p2 = netVisual_circle(cellchat@net$weight, vertex.weight = groupSize,
                 weight.scale = T, label.edge= F,
                 title.name = "Interaction weights/strength",
                 vertex.label.cex = 0.6)

In [ ]:
pdf("circle_plot_p1.pdf", width = 12, height = 12) # 放大画布
print(p1)
dev.off()
pdf("circle_plot_p2.pdf", width = 12, height = 12) # 放大画布
print(p2)
dev.off()


In [ ]:
mat <- cellchat@net$weight
par(mfrow = c(3,3), xpd=TRUE)
for (i in 1:nrow(mat)) {
  mat2 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat2[i, ] <- mat[i, ]
  p1 = netVisual_circle(mat2, vertex.weight = groupSize,
                   weight.scale = T, edge.weight.max = max(mat),
                   title.name = rownames(mat)[i])
}

In [ ]:
# Compute the network centrality scores
cellchat <- netAnalysis_computeCentrality(cellchat, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways
# Visualize the computed centrality scores using heatmap, allowing ready identification of major signaling roles of cell groups
cellchat@netP$pathways
# pathways.show = "CXCL"
# netAnalysis_signalingRole_network(cellchat,
#                                   signaling = pathways.show,
#                                   width = 8, height = 2.5,
#                                   font.size = 10)

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
gg1 <- netAnalysis_signalingRole_scatter(cellchat)+ggtitle("All pathway")

# options(repr.plot.width = 9, repr.plot.height = 9)
pdf("2d_scattor.pdf", width = 9, height = 9) # 放大画布
gg1
dev.off()


In [ ]:
library(ComplexHeatmap)

# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
ht1 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "outgoing")
ht2 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "incoming")

In [ ]:
library(CellChat)
library(ComplexHeatmap)
library(grid)

# 生成热图对象
ht1 <- netAnalysis_signalingRole_heatmap(cellchat, font.size = 8, width = 10, height = 20, pattern = "incoming")

pdf("heatmap_incoming.pdf", width = 8, height = 12)
# 调整行名字体大小和行名显示宽度
draw(
  ht1
)
dev.off()

In [ ]:
### Bubble plot
# show all the significant interactions (L-R pairs) from some cell groups (defined by 'sources.use') to other cell groups (defined by 'targets.use')
pdf("LR_Pairs_Bubble.pdf", width = 10, height = 20)
# options(repr.plot.width = 10, repr.plot.height = 20)
netVisual_bubble(cellchat, sources.use = 1, targets.use = c(2:17), remove.isolate = FALSE)
dev.off()   

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
netVisual_bubble(cellchat, sources.use = 1, targets.use = c(1:17), 
                 signaling = c("PECAM1","CDH5","PTPRM","KIT","EPHB","CADM"), remove.isolate = FALSE)
    #             signaling = c(
    #                 "TNF",
    # "IL1",
    # "CCL",
    # "CXCL",
    # "VEGF",
    # "PDGF",
    # "MHC-I",
    # "MHC-II",
    # "NOTCH"
    # ), remove.isolate = FALSE)

In [ ]:
# 感兴趣的pathway
# Signaling role analysis on the cell-cell communication networks of interest
gg2 <- netAnalysis_signalingRole_scatter(cellchat, signaling = c("PECAM1","CDH5","PTPRM","KIT","EPHB","CADM"))+ggtitle("")

options(repr.plot.width = 9, repr.plot.height = 4)
gg2

In [ ]:
## 当然也支持可视化指定的pathway：
# Signaling role analysis on the cell-cell communication networks of interest
sig1 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "outgoing",signaling = c("TNF",
    "IL1",
    "CCL",
    "CXCL",
    "VEGF",
    "PDGF",
    "MHC-I",
    "MHC-II",
    "NOTCH"
    # "PECAM1","CDH5","PTPRM","KIT","EPHB","CADM"
    ))
sig2 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "incoming",signaling = c("TNF",
    "IL1",
    "CCL",
    "CXCL",
    "VEGF",
    "PDGF",
    "MHC-I",
    "MHC-II",
    "NOTCH"
    # "PECAM1","CDH5","PTPRM","KIT","EPHB","CADM"
    ))